# T0. trainer 和 evaluator 的基本使用

&emsp; 1 &ensp; trainer 和 evaluator 的基本关系
 
&emsp; &emsp; 1.1 &ensp; trainer 和 evaluater 的初始化

&emsp; &emsp; 1.2 &ensp; driver 的含义与使用要求

&emsp; &emsp; 1.3 &ensp; trainer 内部初始化 evaluater

&emsp; 2 &ensp; 使用 fastNLP 搭建 argmax 模型

&emsp; &emsp; 2.1 &ensp; trainer_step 和 evaluator_step

&emsp; &emsp; 2.2 &ensp; trainer 和 evaluator 的参数匹配

&emsp; &emsp; 2.3 &ensp; 示例：argmax 模型的搭建

&emsp; 3 &ensp; 使用 fastNLP 训练 argmax 模型
 
&emsp; &emsp; 3.1 &ensp; trainer 外部初始化的 evaluator

&emsp; &emsp; 3.2 &ensp; trainer 内部初始化的 evaluator 

## 1. trainer 和 evaluator 的基本关系

### 1.1  trainer 和 evaluator 的初始化

在`fastNLP 1.0`中，`Trainer`模块和`Evaluator`模块分别表示 **“训练器”和“评测器”**

&emsp; 对应于之前的`fastNLP`版本中的`Trainer`模块和`Tester`模块，其定义方法如下所示

在`fastNLP 1.0`中，需要注意，在同个`python`脚本中先使用`Trainer`训练，然后使用`Evaluator`评测

&emsp; 非常关键的问题在于**如何正确设置二者的 driver**。这就引入了另一个问题：什么是 `driver`？


```python
trainer = Trainer(
        model=model,                        # 模型基于 torch.nn.Module
        train_dataloader=train_dataloader,  # 加载模块基于 torch.utils.data.DataLoader  
        optimizers=optimizer,               # 优化模块基于 torch.optim.*
        ...
        driver="torch",                     # 使用 pytorch 模块进行训练 
        device='cuda',                      # 使用 GPU：0 显卡执行训练
        ...
    )
...
evaluator = Evaluator(
        model=model,                        # 模型基于 torch.nn.Module
        dataloaders=evaluate_dataloader,    # 加载模块基于 torch.utils.data.DataLoader
        metrics={'acc': Accuracy()},        # 测评方法使用 fastNLP.core.metrics.Accuracy 
        ...
        driver=trainer.driver,              # 保持同 trainer 的 driver 一致
        device=None,
        ...
    )
```

### 1.2  driver 的含义与使用要求

在`fastNLP 1.0`中，**driver**这一概念被用来表示**控制具体训练的各个步骤的最终执行部分**

&emsp; 例如神经网络前向、后向传播的具体执行、网络参数的优化和数据在设备间的迁移等

在`fastNLP 1.0`中，**Trainer 和 Evaluator 都依赖于具体的 driver 来完成整体的工作流程**

&emsp; 具体`driver`与`Trainer`以及`Evaluator`之间的关系之后`tutorial 4`中的详细介绍

注：这里给出一条建议：**在同一脚本中**，**所有的** Trainer **和** Evaluator **使用的** driver **应当保持一致**

&emsp; 尽量不出现，之前使用单卡的`driver`，后面又使用多卡的`driver`，这是因为，当脚本执行至

&emsp; 多卡`driver`处时，会重启一个进程执行之前所有内容，如此一来可能会造成一些意想不到的麻烦

### 1.3 Trainer 内部初始化 Evaluator

在`fastNLP 1.0`中，如果在**初始化 Trainer 时**，**传入参数 evaluator_dataloaders 和 metrics **

&emsp; 则在`Trainer`内部，也会初始化单独的`Evaluator`来帮助训练过程中对验证集的评测

```python
trainer = Trainer(
        model=model,
        train_dataloader=train_dataloader,
        optimizers=optimizer,
        ...
        driver="torch",
        device='cuda',
        ...
        evaluate_dataloaders=evaluate_dataloader,   # 传入参数 evaluator_dataloaders
        metrics={'acc': Accuracy()},                # 传入参数 metrics
        ...
    )
```

## 2. argmax 模型的搭建实例

### 2.1 trainer_step 和 evaluator_step

在`fastNLP 1.0`中，使用`pytorch.nn.Module`搭建需要训练的模型，在搭建模型过程中，除了

&emsp; 添加`pytorch`要求的`forward`方法外，还需要添加 `train_step` 和 `evaluate_step` 这两个方法

```python
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.loss_fn = torch.nn.CrossEntropyLoss()
        pass

    def forward(self, x):
        pass

    def train_step(self, x, y):
        pred = self(x)
        return {"loss": self.loss_fn(pred, y)}

    def evaluate_step(self, x, y):
        pred = self(x)
        pred = torch.max(pred, dim=-1)[1]
        return {"pred": pred, "target": y}
```
***
在`fastNLP 1.0`中，**函数 train_step 是 Trainer 中参数 train_fn 的默认值**

&emsp; 由于，在`Trainer`训练时，**Trainer 通过参数 train_fn 对应的模型方法获得当前数据批次的损失值**

&emsp; 因此，在`Trainer`训练时，`Trainer`首先会寻找模型是否定义了`train_step`这一方法

&emsp; &emsp; 如果没有找到，那么`Trainer`会默认使用模型的`forward`函数来进行训练的前向传播过程

注：在`fastNLP 1.0`中，**Trainer 要求模型通过 train_step 来返回一个字典**，**满足如 {"loss": loss} 的形式**

&emsp; 此外，这里也可以通过传入`Trainer`的参数`output_mapping`来实现输出的转换，详见（trainer的详细讲解，待补充）

同样，在`fastNLP 1.0`中，**函数 evaluate_step 是 Evaluator 中参数 evaluate_fn 的默认值**

&emsp; 在`Evaluator`测试时，**Evaluator 通过参数 evaluate_fn 对应的模型方法获得当前数据批次的评测结果**

&emsp; 从用户角度，模型通过`evaluate_step`方法来返回一个字典，内容与传入`Evaluator`的`metrics`一致

&emsp; 从模块角度，该字典的键值和`metric`中的`update`函数的签名一致，这样的机制在传参时被称为“**参数匹配**”

<img src="./figures/T0-fig-training-structure.png" align="center"></img>

### 2.2 trainer 和 evaluator 的参数匹配

在`fastNLP 1.0`中，参数匹配涉及到两个方面，分别是在

&emsp; 一方面，**在模型的前向传播中**，**dataloader 向 train_step 或 evaluate_step 函数传递 batch**

&emsp; 另方面，**在模型的评测过程中**，**evaluate_dataloader 向 metric 的 update 函数传递 batch**

对于前者，在`Trainer`和`Evaluator`中的参数`model_wo_auto_param_call`被设置为`False`时

&emsp; &emsp; **fastNLP 1.0 要求 dataloader 生成的每个 batch **，**满足如 {"x": x, "y": y} 的形式**

&emsp; 同时，`fastNLP 1.0`会查看模型的`train_step`和`evaluate_step`方法的参数签名，并为对应参数传入对应数值

&emsp; &emsp; **字典形式的定义**，**对应在 Dataset 定义的 \_\_getitem\_\_ 方法中**，例如下方的`ArgMaxDatset`

&emsp; 而在`Trainer`和`Evaluator`中的参数`model_wo_auto_param_call`被设置为`True`时

&emsp; &emsp; `fastNLP 1.0`会将`batch`直接传给模型的`train_step`、`evaluate_step`或`forward`函数

```python
class Dataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, item):
        return {"x": self.x[item], "y": self.y[item]}
```

对于后者，首先要明确，在`Trainer`和`Evaluator`中，`metrics`的计算分为`update`和`get_metric`两步

&emsp; &emsp; **update 函数**，**针对一个 batch 的预测结果**，计算其累计的评价指标

&emsp; &emsp; **get_metric 函数**，**统计 update 函数累计的评价指标**，来计算最终的评价结果

&emsp; 例如对于`Accuracy`来说，`update`函数会更新一个`batch`的正例数量`right_num`和负例数量`total_num`

&emsp; &emsp; 而`get_metric`函数则会返回所有`batch`的评测值`right_num / total_num`

&emsp; 在此基础上，**fastNLP 1.0 要求 evaluate_dataloader 生成的每个 batch 传递给对应的 metric**

&emsp; &emsp; **以 {"pred": y_pred, "target": y_true} 的形式**，对应其`update`函数的函数签名

<img src="./figures/T0-fig-parameter-matching.png" align="center"></img>

### 2.3 示例：argmax 模型的搭建

下文将通过训练`argmax`模型，简单介绍如何`Trainer`模块的使用方式

&emsp; 首先，使用`pytorch.nn.Module`定义`argmax`模型，目标是输入一组固定维度的向量，输出其中数值最大的数的索引

In [1]:
import torch
import torch.nn as nn

class ArgMaxModel(nn.Module):
    def __init__(self, num_labels, feature_dimension):
        nn.Module.__init__(self)
        self.num_labels = num_labels

        self.linear1 = nn.Linear(in_features=feature_dimension, out_features=10)
        self.ac1 = nn.ReLU()
        self.linear2 = nn.Linear(in_features=10, out_features=10)
        self.ac2 = nn.ReLU()
        self.output = nn.Linear(in_features=10, out_features=num_labels)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        pred = self.ac1(self.linear1(x))
        pred = self.ac2(self.linear2(pred))
        pred = self.output(pred)
        return pred

    def train_step(self, x, y):
        pred = self(x)
        return {"loss": self.loss_fn(pred, y)}

    def evaluate_step(self, x, y):
        pred = self(x)
        pred = torch.max(pred, dim=-1)[1]
        return {"pred": pred, "target": y}

&emsp; 接着，使用`torch.utils.data.Dataset`定义`ArgMaxDataset`数据集

&emsp; &emsp; 数据集包含三个参数：维度`feature_dimension`、数据量`data_num`和随机种子`seed`

&emsp; &emsp; 数据及初始化是，自动生成指定维度的向量，并为每个向量标注出其中最大值的索引作为预测标签

In [2]:
from torch.utils.data import Dataset

class ArgMaxDataset(Dataset):
    def __init__(self, feature_dimension, data_num=1000, seed=0):
        self.num_labels = feature_dimension
        self.feature_dimension = feature_dimension
        self.data_num = data_num
        self.seed = seed

        g = torch.Generator()
        g.manual_seed(1000)
        self.x = torch.randint(low=-100, high=100, size=[data_num, feature_dimension], generator=g).float()
        self.y = torch.max(self.x, dim=-1)[1]

    def __len__(self):
        return self.data_num

    def __getitem__(self, item):
        return {"x": self.x[item], "y": self.y[item]}

&emsp; 然后，根据`ArgMaxModel`类初始化模型实例，保持输入维度`feature_dimension`和输出标签数量`num_labels`一致

&emsp; &emsp; 再根据`ArgMaxDataset`类初始化两个数据集实例，分别用来模型测试和模型评测，数据量各1000笔

In [3]:
model = ArgMaxModel(num_labels=10, feature_dimension=10)

train_dataset = ArgMaxDataset(feature_dimension=10, data_num=1000)
evaluate_dataset = ArgMaxDataset(feature_dimension=10, data_num=100)

&emsp; 此外，使用`torch.utils.data.DataLoader`初始化两个数据加载模块，批量大小同为8，分别用于训练和测评

In [4]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
evaluate_dataloader = DataLoader(evaluate_dataset, batch_size=8)

&emsp; 最后，使用`torch.optim.SGD`初始化一个优化模块，基于随机梯度下降法

In [5]:
from torch.optim import SGD

optimizer = SGD(model.parameters(), lr=0.001)

## 3. 使用 fastNLP 1.0 训练 argmax 模型

### 3.1 trainer 外部初始化的 evaluator

通过从`fastNLP`库中导入`Trainer`类，初始化`trainer`实例，对模型进行训练

&emsp; 需要导入预先定义好的模型`model`、对应的数据加载模块`train_dataloader`、优化模块`optimizer`

&emsp; 通过`progress_bar`设定进度条格式，默认为`"auto"`，此外还有`"rich"`、`"raw"`和`None`

&emsp; &emsp; 但对于`"auto"`和`"rich"`格式，在`jupyter`中，进度条会在训练结束后会被丢弃

&emsp; 通过`n_epochs`设定优化迭代轮数，默认为20；全部`Trainer`的全部变量与函数可以通过`dir(trainer)`查询

In [6]:
import sys
sys.path.append('..')

from fastNLP import Trainer

trainer = Trainer(
    model=model,
    driver="torch",
    device='cuda',
    train_dataloader=train_dataloader,
    optimizers=optimizer,
    n_epochs=10,                    # 设定迭代轮数 
    progress_bar="auto"             # 设定进度条格式
)

通过使用`Trainer`类的`run`函数，进行训练

&emsp; 其中，可以通过参数`num_train_batch_per_epoch`决定每个`epoch`运行多少个`batch`后停止，默认全部

&emsp; `run`函数完成后在`jupyter`中没有输出保留，此外，通过`help(trainer.run)`可以查询`run`函数的详细内容

In [7]:
trainer.run()

Output()

通过从`fastNLP`库中导入`Evaluator`类，初始化`evaluator`实例，对模型进行评测

&emsp; 需要导入预先定义好的模型`model`、对应的数据加载模块`evaluate_dataloader`

&emsp; 需要注意的是评测方法`metrics`，设定为形如`{'acc': fastNLP.core.metrics.Accuracy()}`的字典

&emsp; 类似地，也可以通过`progress_bar`限定进度条格式，默认为`"auto"`

In [8]:
from fastNLP import Evaluator
from fastNLP import Accuracy

evaluator = Evaluator(
    model=model,
    driver=trainer.driver,          # 需要使用 trainer 已经启动的 driver
    device=None,
    dataloaders=evaluate_dataloader,
    metrics={'acc': Accuracy()}     # 需要严格使用此种形式的字典
)

通过使用`Evaluator`类的`run`函数，进行训练

&emsp; 其中，可以通过参数`num_eval_batch_per_dl`决定每个`evaluate_dataloader`运行多少个`batch`停止，默认全部

&emsp; 最终，输出形如`{'acc#acc': acc}`的字典，在`jupyter`中，进度条会在评测结束后会被丢弃

In [9]:
evaluator.run()

Output()

{'acc#acc': 0.31, 'total#acc': 100.0, 'correct#acc': 31.0}

{'acc#acc': 0.31, 'total#acc': 100.0, 'correct#acc': 31.0}

### 3.2 trainer 内部初始化的 evaluator 

通过在初始化`trainer`实例时加入`evaluate_dataloaders`和`metrics`，可以实现在训练过程中进行评测

&emsp; 通过`progress_bar`同时设定训练和评估进度条格式，在`jupyter`中，在进度条训练结束后会被丢弃

&emsp; 但是中间的评估结果仍会保留；**通过 evaluate_every 设定评估频率**，可以为负数、正数或者函数：

&emsp; &emsp; **为负数时**，**表示每隔几个 epoch 评估一次**；**为正数时**，**则表示每隔几个 batch 评估一次**

In [10]:
trainer = Trainer(
    model=model,
    driver=trainer.driver,      # 因为是在同个脚本中，这里的 driver 同样需要重用
    train_dataloader=train_dataloader,
    evaluate_dataloaders=evaluate_dataloader,
    metrics={'acc': Accuracy()},
    optimizers=optimizer,
    n_epochs=10,  
    evaluate_every=-1,          # 表示每个 epoch 的结束进行评估
)

通过使用`Trainer`类的`run`函数，进行训练

&emsp; 还可以通过**参数 num_eval_sanity_batch 决定每次训练前运行多少个 evaluate_batch 进行评测**，**默认为 2 **

&emsp; 之所以“先评测后训练”，是为了保证训练很长时间的数据，不会在评测阶段出问题，故作此**试探性评测**

In [11]:
trainer.run()

[18:28:25] INFO     Running evaluator sanity check for 2 batches.              ]8;id=549287;file://../fastNLP/core/controllers/trainer.py\trainer.py]8;;\:]8;id=645362;file://../fastNLP/core/controllers/trainer.py#592\592]8;;\

Output()

Output()

----------------------------- Eval. results on Epoch:1, Batch:0 -----------------------------

{
  "acc#acc": 0.31,
  "total#acc": 100.0,
  "correct#acc": 31.0
}

...

{
  "acc#acc": 0.36,
  "total#acc": 100.0,
  "correct#acc": 36.0
}

----------------------------- Eval. results on Epoch:9, Batch:0 -----------------------------

{
  "acc#acc": 0.37,
  "total#acc": 100.0,
  "correct#acc": 37.0
}

---------------------------- Eval. results on Epoch:10, Batch:0 -----------------------------

{
  "acc#acc": 0.4,
  "total#acc": 100.0,
  "correct#acc": 40.0
}

In [12]:
trainer.evaluator.run()

Output()

{'acc#acc': 0.4, 'total#acc': 100.0, 'correct#acc': 40.0}